# 📙 Zestaw zadań — LangChain: Łańcuchy, Tools oraz Agent ReAct

Zadania odnoszą się do notebooków z przykładami:
- **3_1_LangChain_chains.ipynb**
- **3_2_LangChain_tools.ipynb**
- **3_3_LangChain_ReAct_agent.ipynb**

> **Uzupełnij fragmenty oznaczone `# TODO` / `...`.


## Zadanie 1 — Złóż łańcuch: `Prompt` → `LLM` → `StrOutputParser`

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1) Przygotuj PromptTemplate z jedną zmienną 'query'
prompt = PromptTemplate.from_template("Odpowiedz zwięźle: {query}")  # TODO: zmień tekst, jeśli chcesz

# 2) Przygotuj LLM (nie uruchamiamy zapytań w tym zadaniu)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # TODO: ewentualnie inny model

# 3) Parser tekstowy
parser = StrOutputParser()

# 4) Złóż łańcuch operatorami |
chain = prompt | llm | parser  # TODO

assert chain is not None, "Łańcuch nie został utworzony."

## Zadanie 2 — `Tool`: owiń funkcję w narzędzie

In [ ]:
# Cel: zdefiniuj prostą funkcję i zarejestruj jako narzędzie LangChain.
# Odniesienie: 3_2_LangChain_tools.ipynb

from typing import Annotated
from langchain_core.tools import tool

# 1) Utwórz funkcję, która zwraca długość tekstu
@tool
def text_length(s: Annotated[str, "Tekst wejściowy"]) -> int:
    """Zwraca liczbę znaków w tekście."""
    return len(s)  # TODO: możesz dodać walidację

# 2) Użyj narzędzia bezpośrednio
res = text_length.invoke({"s": "LangChain"})
print("Długość:", res)
assert isinstance(res, int), "Narzędzie powinno zwrócić int."

## Zadanie 3 — Agent ReAct: stwórz szkic agenta korzystającego z narzędzi

In [ ]:
# Cel: Przygotuj minimalny szkic agenta ReAct z jednym narzędziem.
# Odniesienie: 3_3_LangChain_ReAct_agent.ipynb

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # TODO: ewentualnie zmień model

# Lista narzędzi dostępnych dla agenta:
tools = [text_length]  # TODO: możesz dodać kolejne własne narzędzia

# Prosty prompt dla ReAct (instrukcja + format reasoning/tool use)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś agentem ReAct. Myśl krok po kroku i używaj narzędzi tylko gdy to potrzebne."),
    ("human", "{input}")
])

# Utwórz agenta ReAct
agent = create_react_agent(llm, tools, prompt)  # TODO: pozostaje jak jest

# Owiń w executor (bez realnego uruchamiania zadań w tym notatniku)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

assert agent_executor is not None, "Nie udało się utworzyć agenta ReAct."

## Zadanie 4 — Parametryzacja łańcucha: `partial` / stałe wartości

In [ ]:
# Cel: Nadaj części wartości promptu przez partial (jeśli Twoja wersja to wspiera), resztę podaj przy wywołaniu.

from langchain_core.prompts import PromptTemplate

base_prompt = PromptTemplate.from_template("Podsumuj w stylu {style}: {text}")

# Ustal styl domyślny przy pomocy partial (lub zbuduj oddzielny prompt)
try:
    styled_prompt = base_prompt.partial(style="naukowy")  # TODO: zmień styl
    filled = styled_prompt.format(text="Modelowanie tematów w korpusach tekstowych.")
    print(filled)
except Exception as e:
    print("Twoja wersja może nie wspierać partial(). Zastosuj inną technikę:", e)

## Zadanie 5 — Rozszerz narzędzie o walidację i obsługę błędów

In [ ]:
# Cel: rozbuduj narzędzie tak, by bezpiecznie liczyło słowa i obsługiwało puste wejście.
from langchain_core.tools import tool

@tool
def word_count(s: Annotated[str, "Tekst do zliczania słów"]) -> int:
    """Zwraca liczbę słów w tekście. Zwraca 0 gdy tekst jest pusty lub None."""
    if not s:
        return 0
    return len(str(s).split())

print("word_count('Ala ma kota') ->", word_count.invoke({"s": "Ala ma kota"}))
print("word_count('') ->", word_count.invoke({"s": ""}))
assert word_count.invoke({"s": ""}) == 0, "Puste wejście powinno zwrócić 0."
